In [1]:
import numpy as np
import cvxpy as cp
from cvxpy.settings import OPTIMAL

# DATA

In [2]:
G = np.matrix([[1.0,0.1,0.2,0.1,0.0],
               [0.1,1.0,0.1,0.1,0.0],
               [0.2,0.1,2.0,0.2,0.2],
               [0.1,0.1,0.2,1.0,0.1],
               [0.0,0.0,0.2,0.1,1.0]])
n = 5
P_tx_max = 3
P_rx_max = 5
P_grp_max = np.array([4, 6]).reshape(-1, 1)
sigma = 0.5

In [3]:
p = cp.Variable((n, 1))
alpha = cp.Parameter(nonneg=True)

In [4]:
G_diag = np.diag(np.diag(G))
G_nondiag = G - G_diag

total_rx_power = G@p

S = G_diag@p
I = G_nondiag@p

SINR = S/(I + sigma)


group_matrix = np.matrix([[1, 1, 0, 0, 0], [0, 0, 1, 1, 1]])
P_grp_tx = group_matrix@p

In [5]:
Constraints = [
    0 <= p,
    p <= P_tx_max,
    total_rx_power <= P_rx_max*np.ones((5, 1)),
    P_grp_tx <= P_grp_max,
    (I + sigma) <= alpha*S
]
Objective = cp.Minimize(0)

problem = cp.Problem(objective=Objective, constraints=Constraints)

In [6]:
l = 0
u = 1e5
e_0 = 0.05
i = 0
e = u-l

while (l <= u and u - l > e_0):
    alpha.value = 0.5*(l + u)
    print("#ITR:{} e={}, Is Converged ? {}".format(i, u-l, u - l <= e_0))
    
    problem.solve()
    
    if problem.status == OPTIMAL:
        u = alpha.value
    else:
        l = alpha.value
    i += 1
    
    
print("SANITY CHECK:- Is Converged ? {}".format(u - l <= e_0))
print("\n\nTheoretical number of iterations to converge is ceil(log2(e/e_0)) = {}".format(int(np.ceil(np.log2(e/e_0)))))

#ITR:0 e=100000.0, Is Converged ? False
#ITR:1 e=50000.0, Is Converged ? False
#ITR:2 e=25000.0, Is Converged ? False
#ITR:3 e=12500.0, Is Converged ? False
#ITR:4 e=6250.0, Is Converged ? False
#ITR:5 e=3125.0, Is Converged ? False
#ITR:6 e=1562.5, Is Converged ? False
#ITR:7 e=781.25, Is Converged ? False
#ITR:8 e=390.625, Is Converged ? False
#ITR:9 e=195.3125, Is Converged ? False
#ITR:10 e=97.65625, Is Converged ? False
#ITR:11 e=48.828125, Is Converged ? False
#ITR:12 e=24.4140625, Is Converged ? False
#ITR:13 e=12.20703125, Is Converged ? False
#ITR:14 e=6.103515625, Is Converged ? False
#ITR:15 e=3.0517578125, Is Converged ? False
#ITR:16 e=1.52587890625, Is Converged ? False
#ITR:17 e=0.762939453125, Is Converged ? False
#ITR:18 e=0.3814697265625, Is Converged ? False
#ITR:19 e=0.19073486328125, Is Converged ? False
#ITR:20 e=0.095367431640625, Is Converged ? False
SANITY CHECK:- Is Converged ? True


Theoretical number of iterations to converge is ceil(log2(e/e_0)) = 21


In [7]:
print("Tx powers = {}".format(p.value[:, 0]))
print("Maximum Tx power = {} and Limit = {}".format(np.max(p.value[:, 0]), P_tx_max))
print("Tx power of group [1, 2] = {} and Limit = {}".format(P_grp_tx.value[0, 0], P_grp_max[0, 0]))
print("Tx power of group [3, 4, 5] = {} and Limit = {}".format(P_grp_tx.value[1, 0], P_grp_max[1, 0]))
print("\nTotal Rx powers = {} and Limit = {}".format(total_rx_power.value[:, 0], P_rx_max))
print("\nSINR = {}".format(SINR.value[:, 0]))
print("Maximum SINR = {}, Minimum SINR = {}".format(np.max(SINR.value[:, 0]), np.min(SINR.value[:, 0])))

Tx powers = [2.03064307 1.84783351 1.57955944 2.29979494 1.86257673]
Maximum Tx power = 2.299794943421712 and Limit = 3
Tx power of group [1, 2] = 3.8784765829017815 and Limit = 4
Tx power of group [3, 4, 5] = 5.7419311142516465 and Limit = 6

Total Rx powers = [2.76131781 2.43883326 4.58250519 3.18981216 2.40846811] and Limit = 5

SINR = [1.65002418 1.69370664 1.64247759 1.65450824 1.78085101]
Maximum SINR = 1.7808510095478347, Minimum SINR = 1.6424775880720277
